In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import cluster
from sklearn import svm
from sklearn import tree
from sklearn import model_selection

In [2]:
from sklearn import model_selection as ms

In [3]:
200*.8/5

32.0

In [4]:
iris_df = sns.load_dataset('iris')
iris_X = iris_df.drop(columns='species')
iris_y = iris_df['species']

iris_X_train, iris_X_test, iris_y_train, iris_y_test = model_selection.train_test_split(iris_X,iris_y, test_size =.2)

In [5]:
dt = tree.DecisionTreeClassifier()
params_dt = {'criterion':['gini','entropy'],'max_depth':[2,3,4],
       'min_samples_leaf':list(range(2,20,2))}

In [6]:
dt_opt = model_selection.GridSearchCV(dt, params_dt)
dt_opt.fit(iris_X_train,iris_y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4],
                         'min_samples_leaf': [2, 4, 6, 8, 10, 12, 14, 16, 18]})

In [7]:
dt_cv_df = pd.DataFrame(dt_opt.cv_results_)

In [8]:
dt_cv_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002049,0.000196,0.001621,0.000156,gini,2,2,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
1,0.002053,0.000108,0.001571,0.000077,gini,2,4,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
2,0.002058,0.000157,0.001492,0.000082,gini,2,6,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
3,0.002292,0.000314,0.001620,0.000184,gini,2,8,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
4,0.002174,0.000091,0.001536,0.000081,gini,2,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
5,0.002055,0.000145,0.001601,0.000163,gini,2,12,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
6,0.002312,0.000573,0.001642,0.000125,gini,2,14,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
7,0.002100,0.000134,0.001613,0.000256,gini,2,16,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
8,0.002072,0.000079,0.001580,0.000218,gini,2,18,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.833333,1.0,0.916667,0.833333,0.916667,0.900000,0.062361,17
9,0.002489,0.000338,0.001585,0.000112,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.916667,1.0,0.916667,0.833333,0.958333,0.925000,0.055277,3


In [9]:
type(dt_opt.best_estimator_)

sklearn.tree._classes.DecisionTreeClassifier

In [10]:
dt_opt.best_params_

{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 4}

In [11]:
svmclf = svm.SVC()
param_grid_svm = {'kernel':['linear','rbf'], 'C':[.5, 1, 10]}
svm_opt = model_selection.GridSearchCV(svmclf,param_grid_svm)

In [12]:
svm_opt.fit(iris_X_train,iris_y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.5, 1, 10], 'kernel': ['linear', 'rbf']})

In [13]:
svm_opt.best_params_

{'C': 0.5, 'kernel': 'linear'}

In [14]:
svm_opt.score(iris_X_test,iris_y_test), dt_opt.score(iris_X_test,iris_y_test)

(1.0, 1.0)